In [ ]:
!pip install catboost optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 27.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import optuna


In [ ]:
def feature_engineering(df):
    df = df.copy()

    # ===============================
    # 1. 기본 위험 플래그
    # ===============================
    df['비만여부'] = (df['BMI'] >= 25).astype(int)
    df['고혈당'] = (df['공복 혈당'] >= 126).astype(int)
    df['고혈압'] = (df['혈압'] >= 140).astype(int)
    df['고중성지방'] = (df['중성 지방'] >= 150).astype(int)
    df['신장위험'] = (df['혈청 크레아티닌'] >= 1.3).astype(int)

    # ===============================
    # 2. 대사 위험 점수
    # ===============================
    df['대사위험점수'] = (
        df['비만여부'] +
        df['고혈당'] +
        df['고혈압'] +
        df['고중성지방'] +
        df['신장위험']
    )

    # ===============================
    # 3. 위험 강도 분리 (🔥 중요)
    # ===============================
    df['강위험'] = (
        (df['고혈당'] == 1) &
        (df['고혈압'] == 1) &
        (df['비만여부'] == 1)
    ).astype(int)

    df['준위험'] = (
        (df['고혈당'] + df['고혈압'] + df['비만여부']) == 2
    ).astype(int)

    # ===============================
    # 4. 경계 완충 변수 (🔥 분포 조절 핵심)
    # ===============================
    df['혈당_경계'] = (
        (df['공복 혈당'] >= 115) & (df['공복 혈당'] < 126)
    ).astype(int)

    df['BMI_경계'] = (
        (df['BMI'] >= 24) & (df['BMI'] < 25)
    ).astype(int)

    df['혈압_경계'] = (
        (df['혈압'] >= 130) & (df['혈압'] < 140)
    ).astype(int)

    # ===============================
    # 5. 가짜 위험 제거
    # ===============================
    df['단독간효소'] = (
        (df['간 효소율'] > 40) &
        (df['대사위험점수'] <= 1)
    ).astype(int)

    # ===============================
    # 6. 위험 카운트 범주화 (cut)
    # ===============================
    df['위험카운트_bin'] = pd.cut(
        df['대사위험점수'],
        bins=[-1, 1, 2, 3, 5],
        labels=['매우안전', '안전', '주의', '고위험'],
        include_lowest=True

    )

    # ===============================
    # 7. 로그 변환 (연속형 보조)
    # ===============================
    df['혈당_log'] = np.log1p(df['공복 혈당'])
    df['지질_log'] = np.log1p(df['중성 지방'])
    df['혈압_log'] = np.log1p(df['혈압'])

    return df


In [ ]:
train = pd.read_csv("/content/train.csv")
test  = pd.read_csv("/content/test.csv")

train = feature_engineering(train)
test  = feature_engineering(test)

train.isna().sum().sort_values(ascending=False).head()



,0
ID,0
나이,0
키(cm),0
몸무게(kg),0
BMI,0


In [ ]:
X = train.drop(columns=['label','ID'])
y = train['label']

In [ ]:
cat_features = [
    '위험카운트_bin'
]

In [ ]:
X['위험카운트_bin'].dtype

CategoricalDtype(categories=['매우안전', '안전', '주의', '고위험'], ordered=True, categories_dtype=object)

In [ ]:
def objective(trial):

    params = {
        "iterations": trial.suggest_int("iterations", 500, 1200),
        "depth": trial.suggest_int("depth", 4, 7),
        "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.12, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 2, 10),
        "random_strength": trial.suggest_float("random_strength", 0, 2),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),
        "loss_function": "Logloss",
        "eval_metric": "Accuracy",
        "verbose": 0,
        "random_seed": 42
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for tr_idx, val_idx in cv.split(X, y):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        model = CatBoostClassifier(**params)
        model.fit(
            X_tr, y_tr,
            cat_features=cat_features,
            eval_set=(X_val, y_val),
            verbose=False
        )

        preds = model.predict(X_val)
        scores.append(accuracy_score(y_val, preds))

    return np.mean(scores)


In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Best Score:", study.best_value)
print("Best Params:", study.best_params)


[I 2026-01-04 09:37:12,782] A new study created in memory with name: no-name-0652a3d1-5fda-4a3c-be7f-3e2dee7c5062
[I 2026-01-04 09:37:27,476] Trial 0 finished with value: 0.735 and parameters: {'iterations': 547, 'depth': 4, 'learning_rate': 0.02997735956867156, 'l2_leaf_reg': 9.585635826952362, 'random_strength': 1.0510653473187337, 'bagging_temperature': 0.5213224476269793}. Best is trial 0 with value: 0.735.
[I 2026-01-04 09:37:48,478] Trial 1 finished with value: 0.741 and parameters: {'iterations': 683, 'depth': 5, 'learning_rate': 0.05166767797658601, 'l2_leaf_reg': 2.6677981268203608, 'random_strength': 0.32266449271734476, 'bagging_temperature': 0.6897381946193375}. Best is trial 1 with value: 0.741.
[I 2026-01-04 09:38:15,270] Trial 2 finished with value: 0.7404285714285714 and parameters: {'iterations': 742, 'depth': 6, 'learning_rate': 0.036087848433200204, 'l2_leaf_reg': 4.59109720095412, 'random_strength': 1.1634412204653553, 'bagging_temperature': 0.8605881130034201}. Bes

Best Score: 0.7432857142857143
Best Params: {'iterations': 1022, 'depth': 7, 'learning_rate': 0.07258675557879296, 'l2_leaf_reg': 6.713770859055307, 'random_strength': 0.7463462516668624, 'bagging_temperature': 0.9339888729016884}


In [ ]:
best_model = CatBoostClassifier(
    **study.best_params,
    loss_function="Logloss",
    eval_metric="Accuracy",
    random_seed=42,
    verbose=200
)

best_model.fit(X, y, cat_features=cat_features)


0:	learn: 0.7201429	total: 9.43ms	remaining: 9.62s
200:	learn: 0.8525714	total: 1.88s	remaining: 7.67s
400:	learn: 0.9212857	total: 3.79s	remaining: 5.86s
600:	learn: 0.9581429	total: 6.41s	remaining: 4.49s
800:	learn: 0.9771429	total: 8.28s	remaining: 2.28s
1000:	learn: 0.9871429	total: 10.2s	remaining: 214ms
1021:	learn: 0.9881429	total: 10.4s	remaining: 0us


In [ ]:
test = test.drop(columns=['ID'])
proba = best_model.predict_proba(test)[:, 1]
threshold = np.percentile(proba, 68)
test_pred = (proba >= threshold).astype(int)

In [ ]:
pd.Series(test_pred).value_counts(normalize=True)

,proportion
0,0.7
1,0.3


In [ ]:
submit = pd.read_csv('/content/sample_submission.csv')
submit['label'] = test_pred
submit
# 예측한 결과를 파일로 저장합니다. index 인자의 값을 False로 설정하지 않으면 제출이 정상적으로 진행되지 않습니다.
submit.to_csv('submission.csv', index = False)

In [ ]:
print("Train label 비율")
print(y.value_counts(normalize=True))

print("Test 예측 비율")
print(pd.Series(test_pred).value_counts(normalize=True))


Train label 비율
label
0    0.632714
1    0.367286
Name: proportion, dtype: float64
Test 예측 비율
0    0.7
1    0.3
Name: proportion, dtype: float64


In [ ]:
df4 = pd.read_csv('/content/submission.csv')
df4.label.value_counts()